In [1]:
!pip install wandb -qqq
!pip install -q simpletransformers
!pip install requests
!pip install sacremoses
import wandb
import pandas as pd
import sklearn
wandb.login()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 34.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 21.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 206.5/206.5 kB 22.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.7/250.7 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 53.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 34.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 25.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 23.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 27.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score
from simpletransformers.classification import ClassificationModel, ClassificationArgs
from google.colab import drive
import os
drive.mount('/content/drive')
os.chdir('/content/drive/My Drive/')

Mounted at /content/drive


In [3]:
df = pd.read_csv("daneINL.csv", encoding="ISO-8859-1")

def preprocess_file():
  a = df[df.iloc[:, 0] != df.iloc[:, 1]]
  tekst = a.iloc[:, 0].tolist() + a.iloc[:, 1].tolist()
  label = [0] * len(a) + [1] * len(a)
  new_df = pd.DataFrame({'Tekst': tekst, 'Label': label})
  return new_df

df = preprocess_file()
print(df)

train_ratio = 0.75
dev_ratio = 0.15
test_ratio = 0.1

train_df, temp_df = train_test_split(df, test_size=(1 - train_ratio), random_state=32)
eval_df, test_df = train_test_split(temp_df, test_size=test_ratio/(test_ratio + dev_ratio), random_state=32)

model_type = "herbert"
model_name = "allegro/herbert-base-cased"
# model_type = "distilbert"
# model_name = "distilbert-base-uncased"

train_args = {
    'evaluate_during_training': True,
    'num_train_epochs': 4,
    'save_eval_checkpoints': False,
    'train_batch_size': 8,
    'eval_batch_size': 8,
    'overwrite_output_dir': True,
    'wandb_project': "INL_projekt_s20783",
}

model = ClassificationModel(
    model_type,
    model_name,
    use_cuda=False,
    cuda_device=0,
    num_labels=2,
    args=train_args
)

model.train_model(train_df, eval_df=eval_df)
result, model_outputs, wrong_predictions = model.eval_model(test_df)
print(result)


dev_predictions, dev_raw_outputs = model.predict(eval_df.iloc[:,0].tolist())
dev_f1 = f1_score(eval_df.iloc[:,1].tolist(), dev_predictions)
dev_accuracy = accuracy_score(eval_df.iloc[:,1].tolist(), dev_predictions)

print("F1-Score:", dev_f1)
print("Accuracy:", dev_accuracy)


test_predictions, test_raw_outputs = model.predict(test_df.iloc[:,0].tolist())
test_f1 = f1_score(test_df.iloc[:,1].tolist(), test_predictions)
test_accuracy = accuracy_score(test_df.iloc[:,1].tolist(), test_predictions)

print("Test F1-Score:", test_f1)
print("Test Accuracy:", test_accuracy)

               Tekst  Label
0         Nauczyciel      0
1             Lekarz      0
2            Student      0
3         Sprzedawca      0
4           In¿ynier      0
..               ...    ...
223        Hokeistka      1
224        Cielanka      1
225         Krawcowa      1
226  Sêdzi pi³karska      1
227            DJ-ka      1

[228 rows x 2 columns]


Some weights of the model checkpoint at allegro/herbert-base-cased were not used when initializing BertForSequenceClassification: ['cls.sso.sso_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.sso.sso_relationship.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification 

/usr/local/lib/python3.10/dist-packages/simpletransformers/classification/classification_model.py:612: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(


  0%|          | 0/171 [00:00<?, ?it/s]

Epoch:   0%|          | 0/4 [00:00<?, ?it/s]

...

  0%|          | 0/5 [00:00<?, ?it/s]

F1-Score: 1.0
Accuracy: 1.0
167         Prokuratorka
213            Florystka
104    Projektant wnêtrz
72               Stra¿ak
224            Cielanka
121              Pisarka
154             Muzyczka
46       Artysta uliczny
29                Kelner
18              Dentysta
26              Reporter
227                DJ-ka
66                 Nurek
91              Animator
109             Hokeista
136         Nauczycielka
137            Graficzka
34              Fotograf
20               Tancerz
32             Prezydent
127        Informatyczka
132            Dentystka
143             Kelnerka
Name: Tekst, dtype: object


  0%|          | 0/23 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

Test F1-Score: 1.0
Test Accuracy: 1.0
